In [2]:
import os
import numpy as np
import pandas as pd

In [18]:
df_train = pd.read_csv('combo_1and2_train.tsv', sep='\t')
df_valid = pd.read_csv('combo_1and2_valid.tsv', sep='\t')

# we don't have a validation set for what we're trying to do
df_train = pd.concat([df_train, df_valid])

# only keep binders
df_train = df_train[df_train['binder'].values.astype(bool)]

## first, have to find the allele of the experimentally-resolved structures

# filter keeping rows where the value of 'targetid' is of length equal to 4
df_train_crystal = df_train[df_train['targetid'].str.len() == 4]

# copy over the pdbfiles of crystal structures
inputdir = '/Users/gianmarcovisani/Desktop'
outdir = './pdbs/pmhc'
os.makedirs(outdir, exist_ok=True)

for i, row in df_train_crystal.iterrows():
    pdbpath = os.path.join(inputdir, row['native_pdbfile'])

    os.system(f'cp {pdbpath} {outdir}')

    # change its name to be just the pdbid
    old_pdbpath = os.path.join(outdir, os.path.basename(pdbpath))
    old_pdbfile_name = os.path.basename(old_pdbpath)[:-4]
    new_pdbfile_name = old_pdbfile_name[:4]
    new_pdbpath = os.path.join(outdir, new_pdbfile_name+'.pdb')

    os.system(f'mv {old_pdbpath} {new_pdbpath}')


training_pdbs_crystal = df_train_crystal['targetid'].tolist()

df_with_alleles = pd.read_csv('data_classI_MHC_I_structures.txt', sep='\t')

pdb_to_allele = dict(zip(df_with_alleles['PDB'], df_with_alleles['Allele']))

final_training_pdbs = []
final_training_alleles = []
missing_pdbs = 0
for pdb in training_pdbs_crystal:
    if pdb in pdb_to_allele:
        final_training_pdbs.append(pdb)
        final_training_alleles.append(pdb_to_allele[pdb])
    else:
        missing_pdbs += 1

print('Missing pdbs:', missing_pdbs)

# create a dataframe with the experimentally resolved pdbs and their corresponding alleles
df_train_crystal = pd.merge(df_train_crystal, pd.DataFrame({'targetid': final_training_pdbs, 'allele': final_training_alleles}), on=['targetid'])
df_train_crystal = df_train_crystal.loc[:, ['targetid', 'allele', 'native_pdbfile']]

df_train_crystal.to_csv('training_pdbs_alleles.tsv', sep='\t', index=False)


alleles_of_interest = ['A0101', 'A0201', 'A1101', 'A2402', 'B0702', 'B0801', 'B2705', 'B3501', 'B3508', 'B4405', 'H2-Db', 'H2-Kb', 'H2-Ld']

# from collections import Counter
# allele_counts = Counter(final_training_alleles)
# # sort the alleles by their counts
# sorted_allele_counts = sorted(allele_counts.items(), key=lambda x: x[1], reverse=True)
# # print the sorted allele counts
# for allele in alleles_of_interest:
#     if allele in allele_counts:
#         print(f"{allele}: {allele_counts[allele]}")
#     else:
#         print(f"{allele}: 0")


Missing pdbs: 127
